In [1]:
const config = {
    modelPath: './models/lpr-8n.onnx',
    inputWidth: 640,
    inputHeight: 640,
    sourceWidth: 3024,
    sourceHeight: 4032,
    sourceImage: './sample-images/car-with-licenseplate-resized.jpeg'
};

In [2]:
const { JSDOM } = require('jsdom') as typeof import('jsdom');
const { Canvas, createCanvas, Image, ImageData, loadImage } = require('canvas') as typeof import('canvas');
const jo = require('jpeg-autorotate') as typeof import('jpeg-autorotate');

function installDOM() {
  const dom = new JSDOM();
  global.document = dom.window.document;
  // The rest enables DOM image and canvas and is provided by node-canvas
  global.Image = Image;
  global.HTMLCanvasElement = Canvas;
  global.ImageData = ImageData;
  global.HTMLImageElement = Image;
}


installDOM();
let {error, buffer} = await jo.rotate(config.sourceImage, {useExifOrientation: true, quality: 100});
const image = await loadImage(buffer);

console.log('DOM Initialized to allow canvas and image');
  
export { };

DOM Initialized to allow canvas and image


In [3]:
import fs from "fs";
import cv from "@techstark/opencv-js";

const matToFile = (mat, filename) => {
    const imgData = new ImageData( new Uint8ClampedArray(mat.data), mat.cols, mat.rows )
    const canvas = createCanvas(imgData.width, imgData.height);
    const ctx = canvas.getContext('2d');
    ctx.putImageData(imgData, 0,0);
    fs.writeFileSync(filename, canvas.toBuffer('image/jpeg'));
}

export {  }

In [4]:
import cv from "@techstark/opencv-js";

/**
 * Preprocessing image
 * @param {HTMLImageElement} source image source
 * @param {Number} modelWidth model input width
 * @param {Number} modelHeight model input height
 * @return preprocessed image and configs
 */
const preprocessing = (source, modelWidth, modelHeight) => {
    const mat = cv.imread(source); // read from img tag
    const matC4 = new cv.Mat(); // new image matrix
    cv.cvtColor(mat,matC4, cv.COLOR_RGBA2BGR); // RGBA to BGR
    matToFile(matC4, './sample-images/matPad4.jpg')

    const matC3 = new cv.Mat(mat.rows, mat.cols, cv.CV_8UC3); // new image matrix
    cv.cvtColor(mat, matC3, cv.COLOR_BGR2RGB); // RGBA to BGR
    let input, xRatio, yRatio;

    return {input, xRatio, yRatio}

    // padding image to [n x n] dim
    /*
    const maxSize = Math.max(matC3.rows, matC3.cols); // get max size from width and height
    const xPad = maxSize - matC3.cols, // set xPadding
        xRatio = maxSize / matC3.cols; // set xRatio
    const yPad = maxSize - matC3.rows, // set yPadding
        yRatio = maxSize / matC3.rows; // set yRatio
    const matPad = new cv.Mat(); // new mat for padded image
    cv.copyMakeBorder(matC3, matPad, 0, yPad, 0, xPad, cv.BORDER_CONSTANT); // padding black


    const input = cv.blobFromImage(
        matC3,
        1 / 255.0, // normalize
        new cv.Size(modelWidth, modelHeight), // resize to model input size
        new cv.Scalar(0, 0, 0),
        true, // swapRB
        false // crop
    ); // preprocessing image matrix

    //write image to file


    // release mat opencv
    mat.delete();
    matC3.delete();
    matPad.delete();

    return {input, xRatio, yRatio};
    */
};

console.log('functiond defined');

functiond defined


In [5]:
import cv from "@techstark/opencv-js";
const mat = cv.imread(image); // read from img tag
const matC4 = new cv.Mat(); // new image matrix
const matC5 = new cv.Mat(mat.rows, mat.cols, cv.CV_8UC3); // new image matrix
const matC6 = new cv.Mat(mat.rows, mat.cols, cv.CV_8UC3); // new image matrix

mat.channels
cv.cvtColor(mat,matC4, cv.COLOR_BGR2RGBA); // RGBA to BGR  COLOR_BGR2RGBA
cv.resize(matC4, matC5, new cv.Size(config.inputWidth, config.inputHeight)); // resize
//matC4.resize(new cv.Size(config.inputWidth, config.inputHeight))
matToFile(matC5, './sample-images/matPad5.jpg');

cv.cvtColor(matC5,matC6, cv.COLOR_BGRA2RGB)
matToFile(matC6, './sample-images/matPad6.jpg');
/*
const maxSize = Math.max(matC5.rows, matC5.cols); // get max size from width and height
const xPad = maxSize - matC5.cols, // set xPadding
    xRatio = maxSize / matC5.cols; // set xRatio
const yPad = maxSize - matC5.rows, // set yPadding
    yRatio = maxSize / matC5.rows; // set yRatio
const matPad = new cv.Mat(); // new mat for padded image
cv.copyMakeBorder(matC5, matC6, 0, yPad, 0, xPad, cv.BORDER_CONSTANT); // padding black
*/


const input = cv.blobFromImage(
    matC6,
    1 / 255.0, // normalize
    new cv.Size(config.inputWidth, config.inputHeight), // resize to model input size
    new cv.Scalar(0, 0, 0),
    true, // swapRB
    false // crop
); // preprocessing image matrix
console.log('input',input.size());

///preprocessing(image, config.inputWidth, config.inputHeight);


export {}
    

input { width: 3, height: 1 }


In [ ]:
import fs from 'fs'
//mport { BlobPart } from 'cv//'
//load blob from file using fs.readFileSync
var blobData = fs.readFileSync('./sample-images/blob.txt')
// format is text file with each value on a new line, convert to array
var blobArray = blobData.toString().split("\n") as unknown as any[]
let input = new Blob(blobArray)
// format is text file with each value on a new line
console.log('blobArray',blobArray.length);
//const input = new Blob(fs.readFileSync('blob.txt'))

In [ ]:
import { Tensor, InferenceSession } from "onnxruntime-node";
let tensor = new Tensor("float32", input, [1,3,config.inputWidth,config.inputHeight]); // to ort.Tensor

In [6]:
import { Tensor, InferenceSession } from "onnxruntime-node";
import cv from "@techstark/opencv-js"; //"opencv4nodejs";
console.log('cv',input);
let tensor = new Tensor("float32", (input as cv.Mat).data32F, [1,3,config.inputWidth,config.inputHeight]); // to ort.Tensor

cv Mat {}


In [7]:
import { Tensor, InferenceSession } from "onnxruntime-node";
import cv from "@techstark/opencv-js";

const session = await InferenceSession.create(config.modelPath);
let output = await session.run({images:tensor}, { logSeverityLevel: 0 } );
console.log('output',output);

export {}

output {
  output0: h {
    dims: [ 1, 5, 8400 ],
    type: 'float32',
    data: Float32Array(42000) [
       9.190546035766602, 23.902015686035156,  34.78321838378906,
       37.96491622924805,   39.8082160949707,  42.77347183227539,
       43.15708923339844,  50.99407196044922, 56.792911529541016,
        63.7553825378418,     74.35888671875,  85.12030029296875,
       96.78421020507812, 104.43142700195312, 112.82708740234375,
      116.07919311523438, 121.16452026367188,  129.2518310546875,
      137.42308044433594,  149.2342987060547,    163.42822265625,
        182.142333984375, 184.28338623046875,  185.1971435546875,
       186.9749755859375, 192.37918090820312,  199.6177978515625,
           201.951171875, 208.96836853027344, 241.29188537597656,
      244.35263061523438, 250.94882202148438, 268.38775634765625,
       284.7146911621094,  285.8581848144531, 289.38873291015625,
      292.91485595703125,  293.7406005859375, 294.07672119140625,
      301.73419189453125,        314.21

In [ ]:
///
///

In [8]:
//const { Tensor } = ort as typeof import('onnxruntime-node');
import { Tensor } from "onnxruntime-node";

interface ImageMetadata {
    batch: number;
    channels: number;
    width: number;
    height: number;
}



function imageDataToTensor(data, imageMetaData:ImageMetadata): any {
    // 1a. Extract the R, G, and B channels from the data to form a 3D int array
    const [R, G, B] = new Array([], [], []);
    for (let i = 0; i < data.length; i += 4) {
      R.push(data[i]);
      G.push(data[i + 1]);
      B.push(data[i + 2]);
      // 2. skip data[i + 3] thus filtering out the alpha channel
    }
    console.log('channels extracted');
    ///console.log(R);
    //console.log(G);
    //console.log(B);
    // 1b. concatenate RGB ~= transpose [224, 224, 3] -> [3, 224, 224]
//    const transposedData = R.concat(G).concat(B);
    const transposedData = R.concat(G).concat(B);
    console.log('data transpose complete');

    // 3. convert to float32
    let i, l = transposedData.length; // length, we need this for the loop
    const float32Data = new Float32Array(imageMetaData.batch * imageMetaData.channels * imageMetaData.width * imageMetaData.height); // create the Float32Array for output
    for (i = 0; i < l; i++) {
      float32Data[i] = transposedData[i] / 255.0; // convert to float (pixel value range maybe? 255)
    }
    console.log('data converted to float32');
  
    const inputTensor = new Tensor("float32", float32Data, [imageMetaData.batch, imageMetaData.channels, imageMetaData.width, imageMetaData.height]);
    console.log('inputTensor created');
    // inputTensor.data.set(float32Data);
    console.log('inputTensor data set');
    return inputTensor;
  }

console.log('function defined');

function defined


In [9]:
import { Bitmap, Jimp } from '@jimp/core';
const jimp = require("jimp") as typeof import('jimp');

var imageData : Bitmap;

//resize image to 640x640
jimp.read('./sample-images/car-with-licenseplate.jpeg').then((image) => {
  imageData = image.resize(640, 640).bitmap;
  image.resize(640,640).write('./sample-images/car-with-licenseplate-resized-tiny.jpeg');
  console.log("Image resized");
}).catch(err => {
  console.error(err);
});

Promise { <pending> }


Image resized


In [10]:
import { Tensor, InferenceSession } from "onnxruntime-node";
import cv from "@techstark/opencv-js";

const imageMetadata = {
    width: (imageData as Bitmap).width,
    height: (imageData as Bitmap).height,
    channels: 3,
    batch: 1
};
 tensor = imageDataToTensor((imageData as Bitmap).data, imageMetadata);
 output = await session.run({images:tensor}, { logSeverityLevel: 0 } );
console.log('output',output);

export {}

channels extracted
data transpose complete
data converted to float32
inputTensor created
inputTensor data set
output {
  output0: h {
    dims: [ 1, 5, 8400 ],
    type: 'float32',
    data: Float32Array(42000) [
      11.964459419250488,  11.40317153930664, 11.772724151611328,
      20.090736389160156,  41.95893096923828,  59.14415740966797,
       53.87537384033203, 48.173118591308594, 45.515830993652344,
        45.3691520690918, 57.343292236328125,  68.81871032714844,
      115.52296447753906, 115.28607177734375, 124.54100036621094,
      126.21919250488281,  127.9874267578125, 137.27052307128906,
      142.67034912109375, 146.69973754882812,  149.1882781982422,
      159.96739196777344, 181.38278198242188,  196.6282958984375,
      198.09735107421875,  198.4817352294922, 201.08047485351562,
      201.35208129882812,  202.1505584716797, 207.64581298828125,
      214.54132080078125,   221.981201171875,   229.652099609375,
      249.72137451171875,  268.7005615234375, 285.62835693359

In [ ]:
if(0.1 > 0.25){
    console.log('0.1 > 0.25');
}
else{
    console.log('0.1 < 0.25');
}




In [11]:
const outputFormat = {
    classes : output.output0.dims[1] - 4,
    totalElements : output.output0.dims[2],

}
const confThreshold:number = 0.25;

function sigmoid(value) {

    var k = Math.exp(value);
    return k / (1.0 + k);

}

const boxes:any[] = []; // boxes to draw
console.log('outputFormat',JSON.stringify(outputFormat));
const rawData = output.output0.data;
const dims = output.output0.dims;
for (let idx = 0; idx < dims[1]; idx++) { //dims[1    ]
  const data = rawData.slice(idx * dims[2], (idx + 1) * dims[2]); // get rows
  console.log('data',data.length);
  const elements = data.length / dims[1];
  console.log('elements',elements);
  for (let i = 0; i < elements; i++) {
    const box = data.slice(i * dims[1], (i + 1) * dims[1]);
    //console.log('box',JSON.stringify(box));
    const confidence = sigmoid(box[4] as number);
    //console.log('confidence',confidence);
    if (confidence > confThreshold) {
        const scores = box.slice(4) as Int8Array; // classes probability scores
        const maxScore = Math.max(...scores); // maximum probability scores
        const classId = scores.indexOf(maxScore); // class id of maximum probability scores
        if(sigmoid(scores[classId]) > 0.25) 
        {
            /*
            const [x, y, w, h] = [
                (config.inputWidth * ((box[0] as number /100) * (box[2] as number /100))),
                (config.inputHeight * ((box[1]as number /100) * (box[3] as number /100))),
                (config.inputWidth * ((box[2] as number /100))),
                (config.inputHeight * ((box[3]as number /100))),
            ]*/

             
            let x = box[0] as number;
            let y = box[1] as number;
            let w = box[2] as number;
            let h = box[3] as number;
            let left = Math.round((x - 0.5 * w ));
            let top = Math.round((y - 0.5 * h ));
            let width = Math.round(w);
            let height = Math.round(h);
            boxes.push({
                scores: scores,
                classId: classId,
                confidence: confidence,
                bounding: [left, top, width, height],
                toDraw: true
            });
        }
        
        

    /*
   

          let center_x = Math.round((box[0] as number/1000)  * config.inputWidth);
          let center_y = Math.round((box[1] as number/1000) * config.inputHeight);
          let width = Math.round((box[2] as number)   );
          let height = Math.round((box[3] as number)  );
          let left = Math.round(center_x - width / 2);
          let top = Math.round(center_y - height / 2);
          */

         // [left, top, width, height]
         //[x, y, w, h]




          /*
          boxes.push([{x],[y],[w],[h]]);
          boxes.push({
              label: label,
              probability: score,
              bounding: [x, y, w, h], // upscale box
          }); // update boxes to draw later
          */
      }
  }
    

}
const nmsThreshold = 0.4;
let classNum = outputFormat.classes;
console.log('total  boxes',boxes.length );
const indexes = []
let boxNum = boxes.length;
let tmp_boxes = [];
let sorted_boxes = [];
for (let c = 0; c < classNum; ++c) {
    for (let i = 0; i < boxes.length; ++i) {
        tmp_boxes[i] = [boxes[i], i];
    }
    sorted_boxes = tmp_boxes.sort((a, b) => { return (b[0].scores[c] - a[0].scores[c]); });
    for (let i = 0; i < boxNum; ++i) {
        if (sorted_boxes[i][0].scores[c] === 0) continue;
        else {
            for (let j = i + 1; j < boxNum; ++j) {
                if (IOU(sorted_boxes[i][0], sorted_boxes[j][0]) >= nmsThreshold) {
                    boxes[sorted_boxes[j][1]].toDraw = false;
                }
            }
        }
    }
}


function IOU(box1, box2) {
    let bounding1 = box1.bounding;
    let bounding2 = box2.bounding;
    let s1 = bounding1[2] * bounding1[3];
    let s2 = bounding2[2] * bounding2[3];

    let left1 = bounding1[0];
    let right1 = left1 + bounding1[2];
    let left2 = bounding2[0];
    let right2 = left2 + bounding2[2];
    let overlapW = calOverlap([left1, right1], [left2, right2]);

    let top1 = bounding2[1];
    let bottom1 = top1 + bounding1[3];
    let top2 = bounding2[1];
    let bottom2 = top2 + bounding2[3];
    let overlapH = calOverlap([top1, bottom1], [top2, bottom2]);

    let overlapS = overlapW * overlapH;
    let iou =  overlapS / (s1 + s2 + overlapS);
    //console.log('iou',iou);
    return iou;
}

// Calculate the overlap range of two vector
function calOverlap(range1, range2) {
    let min1 = range1[0];
    let max1 = range1[1];
    let min2 = range2[0];
    let max2 = range2[1];

    if (min2 > min1 && min2 < max1) {
        return max1 - min2;
    } else if (max2 > min1 && max2 < max1) {
        return max2 - min1;
    } else {
        return 0;
    }
}
//console.log('boxes',JSON.stringify(boxes));
const filteredBoxes = boxes.filter((box) => (box.toDraw as boolean) === true);
console.log('filteredBoxes',filteredBoxes.length);
console.log('filteredBoxes',filteredBoxes[0]);
console.log('filteredBoxes',filteredBoxes[1]);

function drawBox(box, output) {
    let bounding = box.bounding;
    let left = bounding[0];
    let top = bounding[1];
    let width = bounding[2];
    let height = bounding[3];

    cv.rectangle(output, new cv.Point(left, top), new cv.Point(left + width, top + height),
                         new cv.Scalar(0, 255, 0));
    cv.rectangle(output, new cv.Point(left, top), new cv.Point(left + width, top + 15),
                         new cv.Scalar(255, 255, 255), cv.FILLED);
    let text = `${box.classId}: ${box.confidence.toFixed(4)}`;
    cv.putText(output, text, new cv.Point(left, top + 10), cv.FONT_HERSHEY_SIMPLEX, 0.3,
                             new cv.Scalar(0, 0, 0));
}

//let inputImg = cv.imread('./images/matPad4.jpg');

let ouputImg = new cv.Mat(config.inputWidth, config.inputHeight, cv.CV_8UC3);
cv.cvtColor(matC5, ouputImg, cv.COLOR_BGR2RGBA);
let boxNumX = boxes.length;
for (let i = 0; i < boxNumX; ++i) {
    if (boxes[i].toDraw) {
        drawBox(boxes[i], ouputImg);
    }
}
matToFile(ouputImg, './sample-images/matPad45.jpg');

    

outputFormat {"classes":1,"totalElements":8400}
data 8400
elements 1680
data 8400
elements 1680
data 8400
elements 1680
data 8400
elements 1680
data 8400
elements 1680
total  boxes 8400
filteredBoxes 8374
filteredBoxes {
  scores: Float32Array(1) [ 41.95893096923828 ],
  classId: 0,
  confidence: 1,
  bounding: [ 6, 1, 12, 20 ],
  toDraw: true
}
filteredBoxes {
  scores: Float32Array(1) [ 45.3691520690918 ],
  classId: 0,
  confidence: 1,
  bounding: [ 35, 31, 48, 46 ],
  toDraw: true
}


In [ ]:
import { exit } from "process";

const results = output.output0;
const boxes = [];
//console.log("results.dims:",results.dims);
// looping through output
for (let idx = 0; idx < results.dims[1]; idx++) {
  const data = results.data.slice(idx * results.dims[2], (idx + 1) * results.dims[2]); // get rows
  console.log("data:",data.length);
  /*
  const box = data.slice(0, 4);
  const scores = data.slice(4); // classes probability scores
  const score = Math.max(...scores); // maximum probability scores
  const label =  0; //scores.indexOf(score); // class id of maximum probability scores

  
  const [x, y, w, h] = [
    (box[0] - 0.5 * box[2]) , // upscale left
    (box[1] - 0.5 * box[3]) , // upscale top
    box[2] , // upscale width
    box[3] , // upscale height
  ]; // keep boxes in maxSize range
  

  const [x, y, w, h] = [
    box[0] ,// upscale left
    box[1] , // upscale top
    box[2] , // upscale width
    box[3] , // upscale height
  ]; // keep boxes in maxSize range


  boxes.push({
    label: label,
    probability: score,
    bounding: [x, y, w, h], // upscale box
  }); // update boxes to draw later
    */
  //return boxes;
}



console.log(boxes);